In [1]:
import numpy as np
import os
import signal
import json

from parlai.core.agents import create_agent, create_agent_from_shared, get_agent_module
from parlai.core.worlds import create_task
from parlai.core.params import ParlaiParser
from parlai.core.utils import Timer, round_sigfigs, warn_once
from parlai.core.logs import TensorboardLogger
from parlai.scripts.build_dict import build_dict, setup_args as setup_dict_args
from parlai.core.distributed_utils import (
    sync_object, is_primary_worker, all_gather_list, is_distributed, num_workers
)
from parlai.scripts.build_pytorch_data import get_pyt_dict_file
from parlai.scripts.train_model import *

In [2]:
task = "personachat"
model = "seq2seq"
batch_size = 36
lr = 1e-2
hidden_size = 128
args = f"""-m parlai.scripts.train_model -m {model} 
           -t {task} 
           -mf '/tmp/model' 
           -bs {batch_size} 
           -lr {lr} 
           -hs {hidden_size}"""

In [3]:
# Create dictionary of arguments
opt = setup_args().parse_args(args.split())

[ Main ParlAI Arguments: ] 
[  batchsize: 36 ]
[  datapath: C:\Users\snowy\Documents\6.864\final-project\Persona-Chat-6.864\data ]
[  datatype: train ]
[  download_path: C:\Users\snowy\Documents\6.864\final-project\Persona-Chat-6.864\downloads ]
[  hide_labels: False ]
[  image_mode: raw ]
[  multitask_weights: [1] ]
[  numthreads: 1 ]
[  show_advanced_args: False ]
[  task: personachat ]
[ ParlAI Model Arguments: ] 
[  dict_class: parlai.core.dict:DictionaryAgent ]
[  init_model: None ]
[  model: seq2seq ]
[  model_file: '/tmp/model' ]
[ Training Loop Arguments: ] 
[  dict_build_first: True ]
[  display_examples: False ]
[  eval_batchsize: None ]
[  evaltask: None ]
[  load_from_checkpoint: False ]
[  max_train_time: -1 ]
[  num_epochs: -1 ]
[  save_after_valid: False ]
[  save_every_n_secs: -1 ]
[  validation_cutoff: 1.0 ]
[  validation_every_n_epochs: -1 ]
[  validation_every_n_secs: -1 ]
[  validation_max_exs: -1 ]
[  validation_metric: accuracy ]
[  validation_metric_mode: None ]


In [4]:
# Agent is essentially an object for our 'Seq2Seq' model
agent = create_agent(opt)
# World is a BatchWorld object that stores the data and task at hand
# It contains a list (worlds) of DialogPartnerWorld objects
world = create_task(opt, agent)

[ no model with opt yet at: '/tmp/model'(.opt) ]
Dictionary: loading dictionary from '/tmp/model'.dict
[ num words =  18745 ]
[ Using CUDA ]


C:\Users\snowy\Anaconda3\envs\pytorch\lib\site-packages\torch\nn\_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


[ Loading existing model params from '/tmp/model' ]
[creating task(s): personachat]
[loading fbdialog data:C:\Users\snowy\Documents\6.864\final-project\Persona-Chat-6.864\data\Persona-Chat\personachat\train_self_original.txt]


C:\Users\snowy\Documents\6.864\final-project\Persona-Chat-6.864\parlai\core\torch_agent.py:730: UserWarning: LR scheduler is different from saved. Starting fresh!
  warn_once("LR scheduler is different from saved. Starting fresh!")


In [5]:
len(world.worlds) # = batch_size

36

Below I show how we run matches with "worlds" and "agents". I have made print statements inside parlai.core.worlds and modules to show what's happening internally. 

It seems agent 0 is simply data extraction and agent 1 is the model (receives observation from agent 0). The data is ordered such that batch 1 is persona + first dialogue line and batch two just provides the next observed lines (but as you can see from the input lines being printed for agent 1 the previous dialogue are still in the sequence). I'm pretty sure everything is saved in agent.batch_observations.

You can look in data/train_both_original.txt for the dialogue. 

In [6]:
# This is how we loop through batches
with world:
    b = 0
    while True:
        print("Batch =", b)
        b += 1
        # do one example / batch of examples
        world.parley()
        
        if b > 6:
            break

Batch = 0
Batch = 1
Batch = 2
Batch = 3
Batch = 4
Batch = 5
Batch = 6


In [7]:
# This is where dialogue is stored (same as above but concatenated)
print(world.batch_observations[1][0]['text'])

your persona: i was born in kansas.
your persona: i work on a farm.
your persona: i know how to drive a truck.
your persona: i love chickens.
your persona: i eat eggs for breakfast.
hi how are you today
hi , i just love the farm life !
hi how are you today
i am fine thank you . i cant wait for breakfast !
what will you eat then
i am certainly having eggs ! hopefully my chickens will have eggs .
nice let me have some fresh eggs
i raise the chickens on my farm here in kansas .
nice where do you work
i work right here at home on the family farm in kansas .
do you listen to music
all the time in my truck that i know how to drive .
what kind of truck is it


# Training Models

In [8]:
# All the exciting stuff happens in parlai.scripts.train_model, essentially we can just modify
# parameters in opt (batch size, lr, etc) and/or rewrite the training loop.
trainer = TrainLoop(opt)

[ building dictionary first... ]
[ dictionary already built .]
[ no model with opt yet at: '/tmp/model'(.opt) ]
Dictionary: loading dictionary from '/tmp/model'.dict
[ num words =  18745 ]
[ Using CUDA ]
[ Loading existing model params from '/tmp/model' ]
[creating task(s): personachat]
[loading fbdialog data:C:\Users\snowy\Documents\6.864\final-project\Persona-Chat-6.864\data\Persona-Chat\personachat\train_self_original.txt]
[ training... ]


In [11]:
# We should probably delete the internal print statements ;) 
trainer.train()

[ time:30592.0s total_exs:8719308 epochs:132.68 ] {'exs': 612, 'lr': 0.01, 'num_updates': 182660, 'loss': 6.146, 'token_acc': 0.2338, 'nll_loss': 6.146, 'ppl': 466.8}
[ time:30594.0s total_exs:8720172 epochs:132.69 ] {'exs': 864, 'lr': 0.01, 'num_updates': 182684, 'loss': 141.5, 'token_acc': 0.2485, 'nll_loss': 5.897, 'ppl': 363.8}
[ time:30596.0s total_exs:8721036 epochs:132.7 ] {'exs': 864, 'lr': 0.01, 'num_updates': 182708, 'loss': 140.5, 'token_acc': 0.2506, 'nll_loss': 5.853, 'ppl': 348.4}
[ time:30598.0s total_exs:8721864 epochs:132.71 ] {'exs': 828, 'lr': 0.01, 'num_updates': 182731, 'loss': 133.6, 'token_acc': 0.2512, 'nll_loss': 5.808, 'ppl': 333.1}
[ time:30600.0s total_exs:8722692 epochs:132.73 ] {'exs': 828, 'lr': 0.01, 'num_updates': 182754, 'loss': 134.4, 'token_acc': 0.253, 'nll_loss': 5.841, 'ppl': 344.2}
[ time:30602.0s total_exs:8723556 epochs:132.74 ] {'exs': 864, 'lr': 0.01, 'num_updates': 182778, 'loss': 140.6, 'token_acc': 0.2505, 'nll_loss': 5.857, 'ppl': 349.5}


[ time:30694.0s total_exs:8762616 epochs:133.33 ] {'exs': 864, 'lr': 0.01, 'num_updates': 183863, 'loss': 138.8, 'token_acc': 0.2591, 'nll_loss': 5.783, 'ppl': 324.6}
[ time:30696.0s total_exs:8763516 epochs:133.35 ] {'exs': 900, 'lr': 0.01, 'num_updates': 183888, 'loss': 148.3, 'token_acc': 0.2467, 'nll_loss': 5.938, 'ppl': 379.2}
[ time:30698.0s total_exs:8764416 epochs:133.36 ] {'exs': 900, 'lr': 0.01, 'num_updates': 183913, 'loss': 145.7, 'token_acc': 0.2584, 'nll_loss': 5.829, 'ppl': 340.0}
[ time:30701.0s total_exs:8765280 epochs:133.38 ] {'exs': 864, 'lr': 0.01, 'num_updates': 183937, 'loss': 139.9, 'token_acc': 0.2519, 'nll_loss': 5.837, 'ppl': 342.6}
[ time:30703.0s total_exs:8766180 epochs:133.39 ] {'exs': 900, 'lr': 0.01, 'num_updates': 183962, 'loss': 144.3, 'token_acc': 0.2583, 'nll_loss': 5.777, 'ppl': 322.8}
[ time:30705.0s total_exs:8767080 epochs:133.4 ] {'exs': 900, 'lr': 0.01, 'num_updates': 183987, 'loss': 150.0, 'token_acc': 0.24, 'nll_loss': 5.999, 'ppl': 403.1}
[

[ time:30796.0s total_exs:8806860 epochs:134.01 ] {'exs': 864, 'lr': 0.01, 'num_updates': 185092, 'loss': 142.1, 'token_acc': 0.2484, 'nll_loss': 5.921, 'ppl': 372.6}
[ time:30799.0s total_exs:8807796 epochs:134.02 ] {'exs': 936, 'lr': 0.01, 'num_updates': 185118, 'loss': 151.6, 'token_acc': 0.2556, 'nll_loss': 5.832, 'ppl': 341.1}
[ time:30801.0s total_exs:8808660 epochs:134.04 ] {'exs': 864, 'lr': 0.01, 'num_updates': 185142, 'loss': 140.0, 'token_acc': 0.253, 'nll_loss': 5.832, 'ppl': 340.9}
[ time:30803.0s total_exs:8809596 epochs:134.05 ] {'exs': 936, 'lr': 0.01, 'num_updates': 185168, 'loss': 149.8, 'token_acc': 0.2598, 'nll_loss': 5.765, 'ppl': 318.8}
[ time:30805.0s total_exs:8810424 epochs:134.06 ] {'exs': 828, 'lr': 0.01, 'num_updates': 185191, 'loss': 134.7, 'token_acc': 0.2484, 'nll_loss': 5.858, 'ppl': 350.0}
[ time:30807.0s total_exs:8811288 epochs:134.08 ] {'exs': 864, 'lr': 0.01, 'num_updates': 185215, 'loss': 141.3, 'token_acc': 0.2509, 'nll_loss': 5.89, 'ppl': 361.3}


[ time:30899.0s total_exs:8851068 epochs:134.68 ] {'exs': 936, 'lr': 0.01, 'num_updates': 186320, 'loss': 152.4, 'token_acc': 0.2516, 'nll_loss': 5.861, 'ppl': 351.2}
[ time:30901.0s total_exs:8852004 epochs:134.69 ] {'exs': 936, 'lr': 0.01, 'num_updates': 186346, 'loss': 151.8, 'token_acc': 0.2541, 'nll_loss': 5.839, 'ppl': 343.6}
[ time:30903.0s total_exs:8852868 epochs:134.71 ] {'exs': 864, 'lr': 0.01, 'num_updates': 186370, 'loss': 138.6, 'token_acc': 0.2525, 'nll_loss': 5.773, 'ppl': 321.5}
[ time:30905.0s total_exs:8853732 epochs:134.72 ] {'exs': 864, 'lr': 0.01, 'num_updates': 186394, 'loss': 141.3, 'token_acc': 0.2496, 'nll_loss': 5.889, 'ppl': 361.0}
[ time:30907.0s total_exs:8854632 epochs:134.73 ] {'exs': 900, 'lr': 0.01, 'num_updates': 186419, 'loss': 147.0, 'token_acc': 0.2555, 'nll_loss': 5.881, 'ppl': 358.1}
[ time:30909.0s total_exs:8855532 epochs:134.75 ] {'exs': 900, 'lr': 0.01, 'num_updates': 186444, 'loss': 149.1, 'token_acc': 0.2435, 'nll_loss': 5.966, 'ppl': 389.8

[ time:31001.0s total_exs:8895492 epochs:135.36 ] {'exs': 900, 'lr': 0.01, 'num_updates': 187554, 'loss': 146.5, 'token_acc': 0.2547, 'nll_loss': 5.862, 'ppl': 351.4}
[ time:31003.0s total_exs:8896356 epochs:135.37 ] {'exs': 864, 'lr': 0.01, 'num_updates': 187578, 'loss': 142.8, 'token_acc': 0.2444, 'nll_loss': 5.948, 'ppl': 382.9}
[ time:31005.0s total_exs:8897256 epochs:135.38 ] {'exs': 900, 'lr': 0.01, 'num_updates': 187603, 'loss': 145.6, 'token_acc': 0.2561, 'nll_loss': 5.824, 'ppl': 338.2}
[ time:31007.0s total_exs:8898120 epochs:135.4 ] {'exs': 864, 'lr': 0.01, 'num_updates': 187627, 'loss': 142.6, 'token_acc': 0.2414, 'nll_loss': 5.944, 'ppl': 381.5}
[ time:31009.0s total_exs:8898984 epochs:135.41 ] {'exs': 864, 'lr': 0.01, 'num_updates': 187651, 'loss': 140.5, 'token_acc': 0.247, 'nll_loss': 5.856, 'ppl': 349.4}
[ time:31011.0s total_exs:8899884 epochs:135.42 ] {'exs': 900, 'lr': 0.01, 'num_updates': 187676, 'loss': 146.4, 'token_acc': 0.2517, 'nll_loss': 5.858, 'ppl': 350.0}


...

[ time:37957.0s total_exs:11917656 epochs:181.34 ] {'exs': 828, 'lr': 0.01, 'num_updates': 271503, 'loss': 130.7, 'token_acc': 0.2568, 'nll_loss': 5.687, 'ppl': 294.9}
[ time:37959.0s total_exs:11918520 epochs:181.36 ] {'exs': 864, 'lr': 0.01, 'num_updates': 271527, 'loss': 135.9, 'token_acc': 0.2614, 'nll_loss': 5.664, 'ppl': 288.3}
[ time:37961.0s total_exs:11919456 epochs:181.37 ] {'exs': 936, 'lr': 0.01, 'num_updates': 271553, 'loss': 148.0, 'token_acc': 0.2593, 'nll_loss': 5.692, 'ppl': 296.6}
[ time:37963.0s total_exs:11920320 epochs:181.38 ] {'exs': 864, 'lr': 0.01, 'num_updates': 271577, 'loss': 139.4, 'token_acc': 0.2487, 'nll_loss': 5.807, 'ppl': 332.5}
[ time:37966.0s total_exs:11921184 epochs:181.4 ] {'exs': 864, 'lr': 0.01, 'num_updates': 271601, 'loss': 137.2, 'token_acc': 0.2589, 'nll_loss': 5.721, 'ppl': 305.3}
[ time:37968.0s total_exs:11922084 epochs:181.41 ] {'exs': 900, 'lr': 0.01, 'num_updates': 271626, 'loss': 144.9, 'token_acc': 0.2476, 'nll_loss': 5.799, 'ppl': 

[ time:38057.0s total_exs:11961072 epochs:182.0 ] {'exs': 864, 'lr': 0.01, 'num_updates': 272709, 'loss': 136.0, 'token_acc': 0.2547, 'nll_loss': 5.669, 'ppl': 289.7}
[ time:38059.0s total_exs:11961936 epochs:182.02 ] {'exs': 864, 'lr': 0.01, 'num_updates': 272733, 'loss': 137.8, 'token_acc': 0.2536, 'nll_loss': 5.743, 'ppl': 311.9}
[ time:38061.0s total_exs:11962836 epochs:182.03 ] {'exs': 900, 'lr': 0.01, 'num_updates': 272758, 'loss': 142.0, 'token_acc': 0.256, 'nll_loss': 5.686, 'ppl': 294.7}
[ time:38063.0s total_exs:11963772 epochs:182.04 ] {'exs': 936, 'lr': 0.01, 'num_updates': 272784, 'loss': 148.1, 'token_acc': 0.2536, 'nll_loss': 5.698, 'ppl': 298.4}
[ time:38065.0s total_exs:11964672 epochs:182.06 ] {'exs': 900, 'lr': 0.01, 'num_updates': 272809, 'loss': 143.2, 'token_acc': 0.2557, 'nll_loss': 5.727, 'ppl': 307.1}
[ time:38067.0s total_exs:11965500 epochs:182.07 ] {'exs': 828, 'lr': 0.01, 'num_updates': 272832, 'loss': 131.0, 'token_acc': 0.2594, 'nll_loss': 5.696, 'ppl': 2

[ time:38157.0s total_exs:12004524 epochs:182.66 ] {'exs': 864, 'lr': 0.01, 'num_updates': 273916, 'loss': 135.3, 'token_acc': 0.2663, 'nll_loss': 5.637, 'ppl': 280.7}
[ time:38159.0s total_exs:12005388 epochs:182.68 ] {'exs': 864, 'lr': 0.01, 'num_updates': 273940, 'loss': 139.5, 'token_acc': 0.2474, 'nll_loss': 5.818, 'ppl': 336.3}
[ time:38161.0s total_exs:12006216 epochs:182.69 ] {'exs': 828, 'lr': 0.01, 'num_updates': 273963, 'loss': 131.7, 'token_acc': 0.2513, 'nll_loss': 5.726, 'ppl': 306.7}
[ time:38163.0s total_exs:12007152 epochs:182.7 ] {'exs': 936, 'lr': 0.01, 'num_updates': 273989, 'loss': 148.8, 'token_acc': 0.253, 'nll_loss': 5.724, 'ppl': 306.1}
[ time:38165.0s total_exs:12008016 epochs:182.72 ] {'exs': 864, 'lr': 0.01, 'num_updates': 274013, 'loss': 137.5, 'token_acc': 0.2557, 'nll_loss': 5.727, 'ppl': 306.9}
[ time:38167.0s total_exs:12008916 epochs:182.73 ] {'exs': 900, 'lr': 0.01, 'num_updates': 274038, 'loss': 142.1, 'token_acc': 0.2633, 'nll_loss': 5.683, 'ppl': 2

[ time:38257.0s total_exs:12048120 epochs:183.33 ] {'exs': 900, 'lr': 0.01, 'num_updates': 275127, 'loss': 142.0, 'token_acc': 0.2562, 'nll_loss': 5.679, 'ppl': 292.8}
[ time:38259.0s total_exs:12049056 epochs:183.34 ] {'exs': 936, 'lr': 0.01, 'num_updates': 275153, 'loss': 147.1, 'token_acc': 0.2587, 'nll_loss': 5.66, 'ppl': 287.2}
[ time:38261.0s total_exs:12049956 epochs:183.36 ] {'exs': 900, 'lr': 0.01, 'num_updates': 275178, 'loss': 140.1, 'token_acc': 0.2563, 'nll_loss': 5.604, 'ppl': 271.5}
[ time:38263.0s total_exs:12050820 epochs:183.37 ] {'exs': 864, 'lr': 0.01, 'num_updates': 275202, 'loss': 135.3, 'token_acc': 0.2638, 'nll_loss': 5.639, 'ppl': 281.0}
[ time:38265.0s total_exs:12051720 epochs:183.38 ] {'exs': 900, 'lr': 0.01, 'num_updates': 275227, 'loss': 143.0, 'token_acc': 0.2531, 'nll_loss': 5.719, 'ppl': 304.5}
[ time:38267.0s total_exs:12052656 epochs:183.4 ] {'exs': 936, 'lr': 0.01, 'num_updates': 275253, 'loss': 148.3, 'token_acc': 0.2584, 'nll_loss': 5.706, 'ppl': 3

({'exs': 7801,
  'accuracy': 0.0007691,
  'f1': 0.1531,
  'bleu': 0.001849,
  'lr': 0.01,
  'num_updates': 276197,
  'loss': 1222.0,
  'token_acc': 0.2774,
  'nll_loss': 5.55,
  'ppl': 257.1},
 {'exs': 7512,
  'accuracy': 0.0007987,
  'f1': 0.1587,
  'bleu': 0.002091,
  'lr': 0.01,
  'num_updates': 276197,
  'loss': 1169.0,
  'token_acc': 0.2822,
  'nll_loss': 5.509,
  'ppl': 247.0})

In [12]:
import transformers

In [ ]:
transformers.AutoTokenizer.from_pretrained

In [13]:
transformers.Seq2SeqTrainer

transformers.trainer_seq2seq.Seq2SeqTrainer